# NLP Code Along

For this code along we will build a spam filter! We'll use the various NLP tools we learned about as well as a new classifier, Naive Bayes.

We'll use a classic dataset for this - UCI Repository SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

In [1]:
import findspark
findspark.init('/home/jing/spark-2.1.0-bin-hadoop2.7')

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [4]:
data = spark.read.csv("smsspamcollection/SMSSpamCollection",inferSchema=True,sep='\t')

In [6]:
data.show(5)

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
+----+--------------------+
only showing top 5 rows



In [7]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [9]:
data.show(truncate = False)

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|class|text                                                                                                                                                                                                |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ham  |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...                                                                                     |
|ham  |Ok lar... Joking wif u oni...                                                                                                                                                

## Clean and Prepare the Data

** Create a new length feature: **

In [11]:
from pyspark.sql.functions import length

In [12]:
data = data.withColumn('length',length(data['text'])) # Create a new column

In [13]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [14]:
# Pretty Clear Difference
data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [37]:
SEED = 123 # Set random set

## Feature Transformations

In [15]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer

tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')    #Convert the classs into numerical categories

In [16]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [17]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

### The Model

We'll use Naive Bayes, but feel free to play around with this choice!

In [18]:
from pyspark.ml.classification import NaiveBayes

In [19]:
# Use defaults
nb = NaiveBayes()

### Pipeline

In [20]:
from pyspark.ml import Pipeline

In [21]:
# Pipeline
# Convert Categorical variables to numeric, tokenize => remove stop words => count vectorizer => idf => vector assembler
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,stopremove,count_vec,idf,clean_up])

In [22]:
cleaner = data_prep_pipe.fit(data)

In [24]:
cleaner

PipelineModel_48189dd4f801191568b1

In [29]:
clean_data = cleaner.transform(data)

In [30]:
clean_data.show(5)

+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|class|                text|length|label|          token_text|         stop_tokens|               c_vec|              tf_idf|            features|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|  ham|Go until jurong p...|   111|  0.0|[go, until, juron...|[go, jurong, poin...|(13458,[8,12,33,6...|(13458,[8,12,33,6...|(13459,[8,12,33,6...|
|  ham|Ok lar... Joking ...|    29|  0.0|[ok, lar..., joki...|[ok, lar..., joki...|(13458,[0,26,308,...|(13458,[0,26,308,...|(13459,[0,26,308,...|
| spam|Free entry in 2 a...|   155|  1.0|[free, entry, in,...|[free, entry, 2, ...|(13458,[2,14,20,3...|(13458,[2,14,20,3...|(13459,[2,14,20,3...|
|  ham|U dun say so earl...|    49|  0.0|[u, dun, say, so,...|[u, dun, say, ear...|(13458,[0,73,84,1...|(13458,[0,73,8

### Training and Evaluation!

In [31]:
clean_data.columns

['class',
 'text',
 'length',
 'label',
 'token_text',
 'stop_tokens',
 'c_vec',
 'tf_idf',
 'features']

In [32]:
clean_data = clean_data.select(['label','features'])

In [33]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13459,[8,12,33,6...|
|  0.0|(13459,[0,26,308,...|
|  1.0|(13459,[2,14,20,3...|
|  0.0|(13459,[0,73,84,1...|
|  0.0|(13459,[36,39,140...|
|  1.0|(13459,[11,57,62,...|
|  0.0|(13459,[11,55,108...|
|  0.0|(13459,[133,195,4...|
|  1.0|(13459,[1,50,124,...|
|  1.0|(13459,[0,1,14,29...|
|  0.0|(13459,[5,19,36,4...|
|  1.0|(13459,[9,18,40,9...|
|  1.0|(13459,[14,32,50,...|
|  0.0|(13459,[42,99,101...|
|  0.0|(13459,[567,1745,...|
|  1.0|(13459,[32,113,11...|
|  0.0|(13459,[86,224,47...|
|  0.0|(13459,[0,2,52,13...|
|  0.0|(13459,[0,77,107,...|
|  1.0|(13459,[4,32,35,6...|
+-----+--------------------+
only showing top 20 rows



In [38]:
(training,testing) = clean_data.randomSplit([0.7,0.3], seed=SEED)

In [39]:
spam_predictor = nb.fit(training)

In [40]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [41]:
test_results = spam_predictor.transform(testing)

In [110]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13461,[0,1,2,14,...|[-612.34877984332...|[0.99999999999999...|       0.0|
|  0.0|(13461,[0,1,5,15,...|[-742.97388469249...|[1.0,5.5494439698...|       0.0|
|  0.0|(13461,[0,1,6,16,...|[-1004.8197043274...|[1.0,5.0315468936...|       0.0|
|  0.0|(13461,[0,1,12,34...|[-879.22017540506...|[1.0,1.0023148863...|       0.0|
|  0.0|(13461,[0,1,15,33...|[-216.47131414494...|[1.0,1.1962236837...|       0.0|
|  0.0|(13461,[0,1,16,21...|[-673.71050817005...|[1.0,1.5549413147...|       0.0|
|  0.0|(13461,[0,1,22,26...|[-382.58333036006...|[1.0,1.7564627587...|       0.0|
|  0.0|(13461,[0,1,25,66...|[-1361.5572580867...|[1.0,2.1016772175...|       0.0|
|  0.0|(13461,[0,1,33,46...|[-378.04433557629...|[1.0,6.5844301586...|       0.0|
|  0.0|(13461,[0

In [42]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [43]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting spam was: {}".format(acc))

Accuracy of model at predicting spam was: 0.92555332782218


Not bad considering we're using straight math on text data! Try switching out the classification models! Or even try to come up with other engineered features!

## Great Job!